In [1]:
from argparse import Namespace


import torch
import torch.nn as nn
import torch.nn.functional as F


from src.models.generator import Generator
from src.config import get_parser

In [2]:
# opt = Namespace(
#     dlatent_size=128,
#     in_channel=3,
#     out_channel=32,
#     kernel=3,
#     ker_size=3,
#     # Body
#     num_layer=5,
#     nfc=32,
#     min_nfc=32,
#     padd_size=1, #math.floor(opt.ker_size/2)
#     nc_im=3,  # image # channels
    
# )
# from argparse import Namespace
# from collections import OrderedDict

# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# from src.stylegan2.op import fused_leaky_relu
# from src.stylegan2.Blocks import ModConvLayer


# class ConvBlock(nn.Sequential):
#     def __init__(self, in_channel, out_channel, ker_size, padd, stride):
#         super(ConvBlock,self).__init__()
#         self.add_module('conv',nn.Conv2d(in_channel ,out_channel,kernel_size=ker_size,stride=stride,padding=padd)),
#         self.add_module('norm',nn.BatchNorm2d(out_channel)),
#         self.add_module('LeakyRelu',nn.LeakyReLU(0.2, inplace=True))

#     def weights_init(m):
#         classname = m.__class__.__name__
#         if classname.find('Conv2d') != -1:
#             m.weight.data.normal_(0.0, 0.02)
#         elif classname.find('Norm') != -1:
#             m.weight.data.normal_(1.0, 0.02)
#             m.bias.data.fill_(0)

# # Generator
# # Takes a style and produces a high resolution model            
# class Generator(nn.Module):
#     def __init__(self, opt):
#         super(Generator, self).__init__()
#         self.noise_amp = opt.G_noise_amp
#         blueprint = torch.load(opt.blueprint)
#         self.register_buffer('vertices', blueprint['vertices'])
#         self.register_buffer('normals', blueprint['normals'])
        

#         self.head = ModConvLayer(
#             opt.dlatent_size, 
#             opt.in_channel,
#             opt.out_channel, 
#             opt.ker_size, 
#         )
#         self.body = nn.Sequential()
#         for i in range(opt.num_layer-2):
#             N = int(opt.nfc/pow(2,(i+1)))
#             block = ConvBlock(max(2*N,opt.min_nfc),max(N, opt.min_nfc),opt.ker_size,opt.padd_size,1)
#             self.body.add_module('block%d'%(i+1),block)
        
#         self.tail = nn.Sequential(
#             nn.Conv2d(max(N,opt.min_nfc),opt.nc_im,kernel_size=opt.ker_size,stride =1,padding=opt.padd_size),
#             nn.Tanh()
#         )
    
#     def forward(self, style):
#         vertices = self.vertices.expand(style.size(0), -1, -1, -1)
#         noise = torch.randn_like(vertices) * self.noise_amp
#         x = vertices + noise        
#         x = self.head(x, style)        
#         x = self.body(x)        
#         x = self.tail(x)        
#         return x
    

parser = get_parser()
opt = parser.parse_args(args=[])
opt

G = Generator(opt)
print(G)
img = torch.rand(1, 3, 128, 128)
style = torch.rand(1, opt.dlatent_size)
G(style).shape

Generator(
  (head): ModConvLayer(
    (conv): EqualizedModConv2d(3, 32, 3, upsample=False, downsample=False)
  )
  (body): Sequential(
    (block1): ConvBlock(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (LeakyRelu): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (block2): ConvBlock(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (LeakyRelu): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (block3): ConvBlock(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (LeakyRelu): LeakyReLU(negative_slope=0.2, inplace=True)
    )
  )
  (tail): Sequential(
    (0): Conv2d(32, 3, kernel_size

torch.Size([1, 3, 512, 512])

In [13]:
blueprint = torch.load(opt.blueprint)
blueprint['vertices'].shape, blueprint['vertices'].dtype

(torch.Size([1, 3, 512, 512]), torch.float64)

In [5]:
blueprint['vertices'].permute(2, 0, 1)[None].shape

torch.Size([1, 3, 512, 512])

In [6]:
blueprint = OrderedDict({
    'vertices': blueprint['vertices'].permute(2, 0, 1)[None],
    'normals': blueprint['normals'].permute(2, 0, 1)[None],
})
torch.save(blueprint, './data/blueprint.pt')

In [7]:
x =  torch.rand((1, 2, 12, 12))
x.shape

torch.Size([1, 2, 12, 12])

In [9]:
x.expand_as(torch.rand((3, 2, 12, 12))).shape

torch.Size([3, 2, 12, 12])

In [11]:
x.expand(4, -1, -1, -1).shape

torch.Size([4, 2, 12, 12])

In [14]:
blueprint = OrderedDict({
    'vertices': blueprint['vertices'].float(),
    'normals': blueprint['normals'].float(),
})
torch.save(blueprint, './data/blueprint.pt')